In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

In [0]:
dataM = pd.read_csv("Female-Names.csv")
dataF = pd.read_csv("Male-Names.csv")

In [0]:
df = pd.concat([dataM,dataF])

In [170]:
df.head()

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian


In [0]:
def should_keep(word):
  if(len(word)) > 19:
    return False
  char_set = [' ', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']  
  for ch in word:
    if ch not in set(char_set):
      return False
  return True
  

In [0]:
def clean(word):
  name = str(word)
  name = name.lower()
  if should_keep(name):
    return name
  else:
    return None 
df.name = df.name.apply(lambda word : clean(word))
  

In [173]:
df.head(10)

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian
5,mazida,f,indian
6,pooja,f,indian
7,kajal,f,indian
8,meena,f,indian
9,sonam,f,indian


In [0]:
df = df.dropna()

In [0]:
char_set = [' ', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
char2idx = {}
index = 0
for ch in char_set:
  char2idx[ch] = index
  index+=1

In [0]:
df = df.drop('race',axis=1)
vector_length = 39
max_word_len = 20
words = []
labels= []
for name,gender in df.itertuples(index=False):
  one_hots_word = []
  
  for ch in name:
    vec = np.zeros(vector_length)
    vec[char2idx[ch]] = 1
    one_hots_word.append(vec)
  for _ in range(max_word_len - len(name)):
    vec = np.zeros(vector_length)
    vec[char2idx['END']] = 1
    one_hots_word.append(vec)
  one_hots_word = np.array(one_hots_word)
  words.append(one_hots_word)
  labels.append(gender)

In [0]:
words = np.array(words)

In [178]:
words.shape

(28000, 20, 39)

In [179]:
len(labels)

28000

In [0]:
labels = np.array(labels)
one = OneHotEncoder()
labels_one_hot = one.fit_transform(labels.reshape(-1 , 1)).todense()

In [181]:
model = Sequential()
model.add(LSTM(128,input_shape=(20,39),return_sequences=True))
model.add(LSTM(120))
model.add(Dropout(.3))
model.add(Dense(2,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 20, 128)           86016     
_________________________________________________________________
lstm_16 (LSTM)               (None, 120)               119520    
_________________________________________________________________
dropout_8 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 242       
Total params: 205,778
Trainable params: 205,778
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_test_split?

In [0]:
X_train, X_test, y_train, y_test = train_test_split(words,labels_one_hot , test_size=0.33, random_state=42)

In [186]:
model.fit(X_train,y_train,epochs=5)

W0718 12:17:32.690856 140048109221760 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
18760/18760 [==============================] - 42s 2ms/step - loss: 0.4828 - acc: 0.7654
Epoch 2/5
18760/18760 [==============================] - 40s 2ms/step - loss: 0.3435 - acc: 0.8615
Epoch 3/5
18760/18760 [==============================] - 40s 2ms/step - loss: 0.2897 - acc: 0.8892
Epoch 4/5
18760/18760 [==============================] - 40s 2ms/step - loss: 0.2563 - acc: 0.9059
Epoch 5/5
18760/18760 [==============================] - 40s 2ms/step - loss: 0.2359 - acc: 0.9149


In [188]:
model.evaluate(X_test,y_test)

9240/9240 [==============================] - 5s 510us/step


[0.2293326390254033, 0.9163419913935971]

In [0]:
gur = "gurpreet singh"
gurf = "gurpreet kaur"


In [0]:
cleaned = clean(gur)

In [206]:
cleaned

'gurpreet singh'

In [0]:
def word2vec(urname):
  char_set = [' ', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
  char2idx = {}
  index = 0
  for ch in char_set:
    char2idx[ch] = index
    index+=1
  vector_length = 39
  max_word_len = 20
  words =[]
  one_hots_word = []
  
  for ch in urname:
    vec = np.zeros(vector_length)
    vec[char2idx[ch]] = 1
    one_hots_word.append(vec)
  for _ in range(max_word_len - len(urname)):
    vec = np.zeros(vector_length)
    vec[char2idx['END']] = 1
    one_hots_word.append(vec)
  one_hots_word = np.array(one_hots_word)
  
  return one_hots_word
 
  

In [0]:
cleaned_mat = word2vec("gurpreet kaur")

In [243]:
cleaned_mat.shape

(20, 39)

In [0]:
cleaned_mat = cleaned_mat.reshape(1,20,39)

In [245]:
model.predict(cleaned_mat)

array([[0.9128404 , 0.08715957]], dtype=float32)

In [246]:
np.argmax(model.predict(cleaned_mat))

0

In [247]:
one.categories_

[array(['f', 'm'], dtype='<U1')]